# Preparation

In [ ]:
import pandas as pd
import numpy as np
import pickle
import configparser
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.style.use('ggplot')

from tqdm import tqdm_notebook as tqdm
from glob import glob
pd.set_option('display.max_columns', None)

import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', None)


%matplotlib inline

username= ## insert your username ##
password= ## insert your password ##
database= ## insert your databasename ##
appid = ## insert your appid for KAKEN ##
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'
engine = create_engine(url)

# Identify Causality by Regression Discontinuity

https://archive.md/rXPgN#selection-693.0-698.0

### What Is Regression Discontinuity (RD)?

The RD design compares subjects that are just above and just below the threshold, as shown in the green box in the “Before” scenario of Figure (A). The subjects in the green box are expected to be very similar. In the “After” scenario the right group receives an intervention and has a different result. This jump or discontinuity in outcomes can be interpreted as the consequence of the intervention.


## Analysis 1: QUANTITY df_c_quantity

### Data Importing + Cleaning

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 

c_quantity_0417 = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quantity_0417.csv')
df_c_quantity = c_quantity_0417
df_c_quantity['directcost'] = df_c_quantity['directcost'] / 1000000

In [ ]:
df_c_quantity.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

# Input data
data = [1179, 1359, 1419, 1580, 1703, 1765, 1830, 1880, 1895, 1913, 1932, 1970, 2000, 2633, 2566, 2381, 2276, 2273, 2273, 2284, 2336, 2372, 2374, 2487, 2533]
labels = [i for i in range(1998, 2023)]

# Create a figure with a transparent background and Times New Roman font
plt.rcParams.update({
    "figure.facecolor": "none",
    "font.family": "Times New Roman",
})

# Create a bar plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(labels, data, color='gray')

# Add vertical line and text
plt.axvline(x=2010.47, color='gray', linestyle='dotted')
ax.text(2004.5, 2350, 'System Reformation \nMulti-year Fund introduced \nFY2011 budget: 263.3 billion yen', fontsize=13, color='black',ha='center')

# Set x-axis labels vertically
plt.xticks( rotation=45, fontsize=14)
plt.yticks( fontsize=14)

plt.ylim(750,2750)

# Set background color to transparent
ax.set_facecolor('none')

# Add a title and axis labels
ax.set_title('Trends in KAKENHI Budget', fontsize=20)
ax.set_ylabel('(100 millions of yen)', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)


# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
df_plot= df_c_quantity
# Replace numerical values with corresponding text
df_plot['allocation_niicode'] = df_plot['allocation_niicode'].replace({1: 'Multi-year\n fund', 0: 'Series of \nSingle-year Grants'})

# Convert categorical variable to numerical value
df_plot['allocation_niicode_codes'] = df_plot['allocation_niicode'].astype('category').cat.codes

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(8, 5))

# Set x-axis and y-axis labels and font sizes
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
ax.set_xlabel('Adopted Year of Research Projects ', fontsize=16)
ax.set_ylabel('Conditional Probability of Receiving \n Treatment, Multi-year Fund ', fontsize=16)


# Add a vertical line and text annotation
plt.axvline(x=2010.5, color='r', linestyle='dotted')

# Create a scatter plot with categorical variables
ax.scatter(df_plot.startfiscalyear, df_plot.allocation_niicode_codes, color='gray')

ax.set_title('Treatment Assignment for Regression Discontinuity Design', fontsize=16)

# Set y-axis tick labels to categorical variable values
yticklabels = ['Series of \nSingle-year \nGrants', 'Multi-year\n fund', ]
ax.set_yticks(range(len(yticklabels)))
ax.set_yticklabels(yticklabels)

#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)
#ax.spines['left'].set_visible(False)

# Display the plot
plt.show()


### Data Visualization

In [ ]:
# Create separate data frames for each category
df_se = df_c_quantity[df_c_quantity['field_niicode'] == 'SE']
df_hs = df_c_quantity[df_c_quantity['field_niicode'] == 'HS']
df_bio = df_c_quantity[df_c_quantity['field_niicode'] == 'BIO']
df_int = df_c_quantity[df_c_quantity['field_niicode'] == 'INT']

# Create a data frame for all categories
df_all = pd.concat([df_se, df_hs, df_bio, df_int])

In [ ]:
# Create a figure with 6 subplots arranged in 3 rows and 2 columns
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))

# Define the categories and colors for the plots
categories = ['Science and Engineering', 'Humanities and Social Sciences', 'Biological Sciences', 'Integrated Science and Innovative Science', 'All Fields']
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Loop over the subplots and plot the data
for i in range(3):
    for j in range(2):
        if i*2 + j < 4:
            category = categories[i*2 + j]
            color = colors[i*2 + j]
            if category == 'Science and Engineering':
                df = df_se
            elif category == 'Humanities and Social Sciences':
                df = df_hs
            elif category == 'Biological Sciences':
                df = df_bio
            elif category == 'Integrated Science and Innovative Science':
                df = df_int
            sns.boxplot(x='startfiscalyear', y='journal_count', data=df, ax=axs[i,j], color=color)
            axs[i,j].set_title(category, fontsize=20)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)

        elif i*2 + j == 4:
            sns.boxplot(x='startfiscalyear', y='journal_count', data=df_all, ax=axs[i,j], color=colors[-1])
            axs[i,j].set_title(categories[-1],fontsize=20)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()

## Analysis 1: QUANTITY df_c_quantity

### Data Importing + Cleaning

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 

c_quantity_0417_working = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quantity_0417_workingpaper.csv')
df_c_quantity = c_quantity_0417_working
df_c_quantity['directcost'] = df_c_quantity['directcost'] / 1000000

In [ ]:
df_c_quantity.head()

### Data Visualization

In [ ]:
# Create separate data frames for each category
df_se = df_c_quantity[df_c_quantity['field_niicode'] == 'SE']
df_hs = df_c_quantity[df_c_quantity['field_niicode'] == 'HS']
df_bio = df_c_quantity[df_c_quantity['field_niicode'] == 'BIO']
df_int = df_c_quantity[df_c_quantity['field_niicode'] == 'INT']

# Create a data frame for all categories
df_all = pd.concat([df_se, df_hs, df_bio, df_int])

In [ ]:
# Create a figure with 6 subplots arranged in 3 rows and 2 columns
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))

# Define the categories and colors for the plots
categories = ['Science and Engineering', 'Humanities and Social Sciences', 'Biological Sciences', 'Integrated Science and Innovative Science', 'All Fields']
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Loop over the subplots and plot the data
for i in range(3):
    for j in range(2):
        if i*2 + j < 4:
            category = categories[i*2 + j]
            color = colors[i*2 + j]
            if category == 'Science and Engineering':
                df = df_se
            elif category == 'Humanities and Social Sciences':
                df = df_hs
            elif category == 'Biological Sciences':
                df = df_bio
            elif category == 'Integrated Science and Innovative Science':
                df = df_int
            sns.boxplot(x='startfiscalyear', y='journal_count', data=df, ax=axs[i,j], color=color)
            axs[i,j].set_title(category, fontsize=20)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)

        elif i*2 + j == 4:
            sns.boxplot(x='startfiscalyear', y='journal_count', data=df_all, ax=axs[i,j], color=colors[-1])
            axs[i,j].set_title(categories[-1],fontsize=20)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()

## Linear Regression (3 models)

### Model 1: simple 

In [ ]:
def fit_linear_regression(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode", data=df).fit()
    return model

df_list = [df_all, df_se, df_hs, df_bio, df_int]
df_names = ["df_all", "df_se", "df_hs", "df_bio", "df_int"]
variables = ["allocation_niicode"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list):
    model = fit_linear_regression(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), round(model.pvalues[j+1], 3)]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value"])
display(table)

In [ ]:
model = smf.ols(formula="journal_count ~ allocation_niicode", data=df_int).fit()
display(model.summary())

In [ ]:
df_all = df_all.sort_values(by ='allocation_niicode')
df= df_all
re = smf.ols(formula="journal_count ~ allocation_niicode", data=df).fit()

plt.figure(figsize=(15, 10), num=None, dpi=80, facecolor='w', edgecolor='k')
sns.scatterplot(data=df, x='startfiscalyear', y='journal_count', hue='allocation_niicode')
l=df.loc[df['allocation_niicode']==0,'allocation_niicode'].count()
plt.axvline(x=2010.5, linestyle='--', color='gray')

plt.plot(df['startfiscalyear'][0:(l-1)], re.predict()[0:(l-1)], '-', color="blue")
plt.plot(df['startfiscalyear'][l:], re.predict()[l:], '-', color="orange")
plt.title("Regression Discontinuity: Before and After the Cutoff", fontsize= 14)

plt.xlabel('Start Fiscal Year of Project', fontsize = 14)
plt.ylabel('Number of Peer-Reviewed Articles\nPublished per Project', fontsize = 14)


### Model2: extended

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

def fit_linear_regression_extended(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode + project_length + directcost", data=df).fit()
    return model

df_list = [df_all, df_se, df_hs, df_bio, df_int]
df_names = ["df_all", "df_se", "df_hs", "df_bio", "df_int"]
variables = ["allocation_niicode", "project_length", "directcost"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list):
    model = fit_linear_regression_extended(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), round(model.pvalues[j+1], 3)]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value"])
display(table)


### Model3 dummy

In [ ]:
dummies = pd.get_dummies(df_all['field_niicode'])
df_all_dummy = df_all
df_all_dummy = pd.concat([df_all_dummy, dummies], axis=1)
df_all_dummy

def fit_linear_regression_extended(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode + project_length + directcost + BIO + HS + INT + SE", data=df).fit()

    return model

model = fit_linear_regression_extended(df_all_dummy)
print(model.summary())

In [ ]:
corr_matrix = df_all_dummy.corr()
# create a heatmap of the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

### Model 4 Polynomial

In [ ]:
def fit_poly_regression(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode  + np.power(startfiscalyear - 2010.5, 2)", data=df).fit()
    return model

df_list = [df_all, df_se, df_hs, df_bio, df_int]
df_names = ["df_all", "df_se", "df_hs", "df_bio", "df_int"]
variables = ["allocation_niicode"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list):
    model = fit_poly_regression(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), model.bse[j+1], round(model.pvalues[j+1], 5)]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value"])
display(table)

## Different bandwidth

In [ ]:
threshold = 2010.5
bandwidth = 7

df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
df_hs_short= df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])
#
#
## Create a figure with 6 subplots arranged in 3 rows and 2 columns
#fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))
#
## Define the categories and colors for the plots
#categories = ['Science and Engineering', 'Humanities and Social Sciences', 'Biological Sciences', 'Integrated Science and Innovative Science', 'All Fields']
#colors = ['blue', 'green', 'red', 'purple', 'orange']
#
## Loop over the subplots and plot the data
#for i in range(3):
#    for j in range(2):
#        if i*2 + j < 4:
#            category = categories[i*2 + j]
#            color = colors[i*2 + j]
#            if category == 'Science and Engineering':
#                df = df_se_short
#            elif category == 'Humanities and Social Sciences':
#                df = df_hs_short
#            elif category == 'Biological Sciences':
#                df = df_bio_short
#            elif category == 'Integrated Science and Innovative Science':
#                df = df_int_short
#            sns.boxplot(x='startfiscalyear', y='journal_count', data=df, ax=axs[i,j], color=color)
#            axs[i,j].set_title(category, fontsize=20)
#            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
#            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)
#
#        elif i*2 + j == 4:
#            sns.boxplot(x='startfiscalyear', y='journal_count', data=df_all_short, ax=axs[i,j], color=colors[-1])
#            axs[i,j].set_title(categories[-1],fontsize=20)
#            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
#            axs[i,j].set_ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize=16)
#
## Adjust the layout of the subplots
#plt.tight_layout()
#
## Show the plot
#plt.show()

## Linear Regression (3 models)

In [ ]:
threshold = 2010.5
bandwidth = 3

df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
df_hs_short= df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])

def fit_linear_regression(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode", data=df).fit()
    return model

df_list_short = [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]
df_names = ["df_all_short", "df_se_short", "df_hs_short", "df_bio_short", "df_int_short"]
variables = ["allocation_niicode"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list_short):
    model = fit_linear_regression(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), round(model.pvalues[j+1], 3), model.nobs, round(model.rsquared, 3)]
        results.append(row)

table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value", "nobs", "rsquared"])
display(table)


### Model 1: simple 

In [ ]:
df_se_short = df_se_short.sort_values(by ='allocation_niicode')
df= df_se_short
re = smf.ols(formula="journal_count ~ allocation_niicode", data=df).fit()

plt.figure(figsize=(15, 10), num=None, dpi=80, facecolor='w', edgecolor='k')
sns.scatterplot(data=df, x='startfiscalyear', y='journal_count', hue='allocation_niicode')
l=df.loc[df['allocation_niicode']==0,'allocation_niicode'].count()
plt.axvline(x=2010.5, linestyle='--', color='gray')

plt.plot(df['startfiscalyear'][0:(l-1)], re.predict()[0:(l-1)], '-', color="blue")
plt.plot(df['startfiscalyear'][l:], re.predict()[l:], '-', color="orange")
plt.title("Regression Discontinuity: Before and After the Cutoff", fontsize= 14)

plt.xlabel('Start Fiscal Year of Project', fontsize = 14)
plt.ylabel('Number of Peer-Reviewed Articles\nPublished per Project', fontsize = 14)


### Model2: extended

In [ ]:

def fit_linear_regression_extended(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode + project_length + directcost", data=df).fit()
    return model

df_list_short = [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]
df_names = ["df_all_short", "df_se_short", "df_hs_short", "df_bio_short", "df_int_short"]
variables = ["allocation_niicode", "project_length", "directcost"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list_short):
    model = fit_linear_regression_extended(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), round(model.pvalues[j+1], 3)]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value"])
display(table)


In [ ]:
threshold = 2010.5
bandwidth = 7

df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
df_hs_short= df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])

def fit_linear_regression_extended(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode + project_length + directcost", data=df).fit()
    return model

df_list_short = [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]
df_names = ["df_all_short", "df_se_short", "df_hs_short", "df_bio_short", "df_int_short"]
variables = ["allocation_niicode", "project_length", "directcost"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list_short):
    model = fit_linear_regression_extended(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), model.nobs, round(model.rsquared, 3),round(model.pvalues[j+1], 3) ]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std",  "nobs", "rsquared","p-value",])
table.sort_values(by ='independent_var')

### Model3 dummy

In [ ]:
dummies = pd.get_dummies(df_all['field_niicode'])
df_all_dummy = df_all_short
df_all_dummy = pd.concat([df_all_dummy, dummies], axis=1)
df_all_dummy

def fit_linear_regression_extended(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode + project_length + directcost + BIO + HS + INT + SE", data=df).fit()

    return model

model = fit_linear_regression_extended(df_all_dummy)
display(model.summary())

## Polynomial

In [ ]:
def fit_poly_regression(df):
    model = smf.ols(formula="journal_count ~ allocation_niicode  + np.power(startfiscalyear - 2010.5, 2)", data=df).fit()
    return model

df_list_short = [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]
df_names = ["df_all_short", "df_se_short", "df_hs_short", "df_bio_short", "df_int_short"]
variables = ["allocation_niicode"]
dependent_var = "journal_count"

results = []
for i, df in enumerate(df_list_short):
    model = fit_poly_regression(df)
    for j, var in enumerate(variables):
        row = [df_names[i], var, dependent_var, round(model.params[j+1], 3), round(model.bse[j+1], 3), round(model.pvalues[j+1], 3)]
        results.append(row)
    
table = pd.DataFrame(results, columns=["df_name", "independent_var", "dependent_var", "coeff", "std", "p-value"])
display(table)

## Analysis 1: QUALITY df_c_quality

### Data Analysis

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 
df_c_quality = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quality_0912_final.csv')
df_c_quality['directcost'] = df_c_quality['directcost'] / 1000000

In [ ]:
df_c_quality = df_c_quality.rename(columns={'Average per Year': 'ave_year','Total Citations': 'total_citations'})
df_c_quality.head(2)


### dataviz

In [ ]:
# Create separate data frames for each category
df_se = df_c_quality[df_c_quality['field_niicode'] == 'SE']
df_hs = df_c_quality[df_c_quality['field_niicode'] == 'HS']
df_bio = df_c_quality[df_c_quality['field_niicode'] == 'BIO']
df_int = df_c_quality[df_c_quality['field_niicode'] == 'INT']

# Create a data frame for all categories
df_all = pd.concat([df_se, df_hs, df_bio, df_int])

In [ ]:
# Create a figure with 6 subplots arranged in 3 rows and 2 columns
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15, 25))

# Define the categories and colors for the plots
categories = ['Science and Engineering', 'Humanities and Social Sciences', 'Biological Sciences', 'Integrated Disciplines', 'All Fields']
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Loop over the subplots and plot the data
for i in range(3):
    for j in range(2):
        if i*2 + j < 4:
            category = categories[i*2 + j]
            color = colors[i*2 + j]
            if category == 'Science and Engineering':
                df = df_se
            elif category == 'Humanities and Social Sciences':
                df = df_hs
            elif category == 'Biological Sciences':
                df = df_bio
            elif category == 'Integrated Disciplines':
                df = df_int
            sns.boxplot(x='startfiscalyear', y='ave_5', data=df, ax=axs[i,j], color=color)
            axs[i,j].set_title(category, fontsize=24)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("ave5", fontsize=16)

        elif i*2 + j == 4:
            sns.boxplot(x='startfiscalyear', y='ave_5', data=df_all, ax=axs[i,j], color=colors[-1])
            axs[i,j].set_title(categories[-1],fontsize=24)
            axs[i,j].set_xlabel("Start Fiscal Year", fontsize=16)
            axs[i,j].set_ylabel("Ave year", fontsize=16)

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()


### Linear Regression

In [ ]:
# Create separate data frames for each category

df_se = df_c_quality[df_c_quality['field_niicode'] == 'SE']
df_hs = df_c_quality[df_c_quality['field_niicode'] == 'HS']
df_bio = df_c_quality[df_c_quality['field_niicode'] == 'BIO']
df_int = df_c_quality[df_c_quality['field_niicode'] == 'INT']

# Create a data frame for all categories
df_all = pd.concat([df_se, df_hs, df_bio, df_int])

### Model 1

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

threshold = 2010.5
bandwidth = 2
print(threshold -bandwidth)


# Subset the data frames
df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
print()
df_hs_short= df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])

# Fit the linear regression models and store the results in a list
results = []
for df in [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]:
    model_year = smf.ols(formula="ave_year ~ allocation_niicode", data=df).fit()
    model_3 = smf.ols(formula="ave_3 ~ allocation_niicode", data=df).fit()
    model_5 = smf.ols(formula="ave_5 ~ allocation_niicode", data=df).fit()
    results.append([model_year.params['allocation_niicode'], model_year.bse['allocation_niicode'], model_year.pvalues['allocation_niicode'], model_year.nobs, model_year.rsquared,
                    model_3.params['allocation_niicode'], model_3.bse['allocation_niicode'], model_3.pvalues['allocation_niicode'], model_3.nobs, model_3.rsquared,
                    model_5.params['allocation_niicode'], model_5.bse['allocation_niicode'], model_5.pvalues['allocation_niicode'], model_5.nobs, model_5.rsquared])

# Create the DataFrame with the results
df_results = pd.DataFrame(results, columns=['Coeff_Year', 'StdErr_Year', 'Pval_Year', 'nobs_Year', 'rsquared_Year', 
                                             'Coeff_3yr', 'StdErr_3yr', 'Pval_3yr', 'nobs_3yr', 'rsquared_3yr',
                                             'Coeff_5yr', 'StdErr_5yr', 'Pval_5yr', 'nobs_5yr', 'rsquared_5yr'])

# Set the row index to the data frame names
df_results.index = [ 'ALL','SE', 'HS', 'BIO', 'INT']

# Display the results
display(df_results)


### Model2

In [ ]:

# Define threshold and bandwidth
threshold = 2010.5
bandwidth = 2

# Subset the data frames
df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
df_hs_short = df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])

# Fit the linear regression models and store the results in a list
results = []
for df, name in zip([df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short], ['ALL', 'SE', 'HS', 'BIO', 'INT']):
    model_year = smf.ols(formula="ave_year ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_3yr = smf.ols(formula="ave_3 ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_5yr = smf.ols(formula="ave_5 ~ allocation_niicode + project_length + directcost", data=df).fit()
    results.append([name,
                    model_year.params['allocation_niicode'],  model_year.bse['allocation_niicode'],
                    model_year.params['project_length'],  model_year.bse['project_length'],
                    model_year.params['directcost'],  model_year.bse['directcost'],
                    model_year.pvalues['allocation_niicode'],model_year.pvalues['project_length'],model_year.pvalues['directcost'],
                    model_year.nobs,  model_year.rsquared,
                    model_3yr.params['allocation_niicode'],  model_3yr.bse['allocation_niicode'],
                    model_3yr.params['project_length'],  model_3yr.bse['project_length'],
                    model_3yr.params['directcost'],  model_3yr.bse['directcost'],
                    model_3yr.pvalues['allocation_niicode'],model_3yr.pvalues['project_length'],model_year.pvalues['directcost'],
                    model_3yr.nobs,  model_year.rsquared,
                    model_5yr.params['allocation_niicode'],  model_5yr.bse['allocation_niicode'],
                    model_5yr.params['project_length'],  model_5yr.bse['project_length'],
                    model_5yr.params['directcost'],  model_5yr.bse['directcost'],
                    model_5yr.pvalues['allocation_niicode'],model_5yr.pvalues['project_length'],model_5yr.pvalues['directcost'],
                    model_5yr.nobs,  model_5yr.rsquared])

# Create the DataFrame with the results
df_results = pd.DataFrame(results, columns=['Data Frame',
                                             'Coeff Year d', 'Std Err Year d', 
                                             'Coeff Year L', 'Std Err Year L', 
                                             'Coeff Year C', 'Std Err Year C', 
                                             'P-value Year d','P-value Year L','P-value Year C',
                                              'nobs Year', 'R-squared Year', 
                                             'Coeff Year3 d', 'Std Err Year3 d', 
                                             'Coeff Year3 L', 'Std Err Year3 L', 
                                             'Coeff Year3 C', 'Std Err Year3 C', 
                                             'P-value Year3 d','P-value Year3 L','P-value Year3 C',
                                              'nobs Year3', 'R-squared Year3', 
                                             'Coeff Year5 d', 'Std Err Year5 d', 
                                             'Coeff Year5 L', 'Std Err Year5 L', 
                                             'Coeff Year5 C', 'Std Err Year5 C', 
                                             'P-value Year5 d','P-value Year5 L','P-value Year5 C',
                                              'nobs Year5', 'R-squared Year5', ])

# Set the row index to the data frame names
df_results = df_results.set_index('Data Frame')

# Display the results
display(df_results.round(3))
#model_year.summary()


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

threshold = 2010.5
bandwidth = 2
print(threshold -bandwidth)


# Subset the data frames
df_se_short = df_se[(df_se['startfiscalyear'] >= threshold - bandwidth) & (df_se['startfiscalyear'] < threshold + bandwidth)]
print()
df_hs_short= df_hs[(df_hs['startfiscalyear'] >= threshold - bandwidth) & (df_hs['startfiscalyear'] < threshold + bandwidth)]
df_bio_short = df_bio[(df_bio['startfiscalyear'] >= threshold - bandwidth) & (df_bio['startfiscalyear'] < threshold + bandwidth)]
df_int_short = df_int[(df_int['startfiscalyear'] >= threshold - bandwidth) & (df_int['startfiscalyear'] < threshold + bandwidth)]
df_all_short = pd.concat([df_se_short, df_hs_short, df_bio_short, df_int_short])

# Fit the linear regression models and store the results in a list
results = []
for df in [df_all_short, df_se_short, df_hs_short, df_bio_short, df_int_short]:
    model_year = smf.ols(formula="ave_year ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_3 = smf.ols(formula="ave_3 ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_5 = smf.ols(formula="ave_5 ~ allocation_niicode + project_length + directcost", data=df).fit()
    results.append([model_year.params['allocation_niicode'], model_year.bse['allocation_niicode'], model_year.pvalues['allocation_niicode'], model_year.nobs, model_year.rsquared,
                    model_3.params['allocation_niicode'], model_3.bse['allocation_niicode'], model_3.pvalues['allocation_niicode'], model_3.nobs, model_3.rsquared,
                    model_5.params['allocation_niicode'], model_5.bse['allocation_niicode'], model_5.pvalues['allocation_niicode'], model_5.nobs, model_5.rsquared])

# Create the DataFrame with the results
df_results = pd.DataFrame(results, columns=['Coeff_Year', 'StdErr_Year', 'Pval_Year', 'nobs_Year', 'rsquared_Year', 
                                             'Coeff_3yr', 'StdErr_3yr', 'Pval_3yr', 'nobs_3yr', 'rsquared_3yr',
                                             'Coeff_5yr', 'StdErr_5yr', 'Pval_5yr', 'nobs_5yr', 'rsquared_5yr'])

# Set the row index to the data frame names
df_results.index = [ 'ALL','SE', 'HS', 'BIO', 'INT']

# Display the results
display(df_results)


In [ ]:
import statsmodels.formula.api as smf

def fit_linear_regression(df):
    # Fit a linear regression model
    model_year = smf.ols(formula="ave_year ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_3 = smf.ols(formula="ave_3 ~ allocation_niicode + project_length + directcost", data=df).fit()
    model_5 = smf.ols(formula="ave_5 ~ allocation_niicode + project_length + directcost", data=df).fit()
    # Return the summary as a string
    display(model_year.summary())
    display(model_3.summary())
    display(model_5.summary())

#df_all_summary = fit_linear_regression(df_all)
#df_se_summary = fit_linear_regression(df_se)
#df_hs_summary = fit_linear_regression(df_hs)
#df_bio_summary = fit_linear_regression(df_bio)
#df_int_summary = fit_linear_regression(df_int)

#df_all_summary = fit_linear_regression(df_all_short)
#df_se_summary = fit_linear_regression(df_se_short)
#df_hs_summary = fit_linear_regression(df_hs_short)
#df_bio_summary = fit_linear_regression(df_bio_short)
df_int_summary = fit_linear_regression(df_int_short)

#re = smf.ols(formula = "journal_count ~ allocation_niicode ", data = df_all).fit()
#print(re.summary())

In [ ]:
def fit_linear_regression(df, df_name):
    # Fit a linear regression model
    model_year = smf.ols(formula="ave_year ~ allocation_niicode", data=df).fit()
    model_3 = smf.ols(formula="ave_3 ~ allocation_niicode", data=df).fit()
    model_5 = smf.ols(formula="ave_5 ~  allocation_niicode", data=df).fit()
    # Return the coefficients, standard errors, t-values, and p-values as a dataframe
    results = pd.DataFrame({
        'DataFrame': [df_name, df_name, df_name],
        'Dependent Variable': ['ave_year', 'ave_3', 'ave_5'],
        'Independent Variable': ['allocation_niicode', 'allocation_niicode', 'allocation_niicode'],
        'Coefficient': [model_year.params['allocation_niicode'], model_3.params['allocation_niicode'], model_5.params['allocation_niicode']],
        'Standard Error': [model_year.bse['allocation_niicode'], model_3.bse['allocation_niicode'], model_5.bse['allocation_niicode']],
        'p-value': [model_year.pvalues['allocation_niicode'], model_3.pvalues['allocation_niicode'], model_5.pvalues['allocation_niicode']],
        't-value': [model_year.tvalues['allocation_niicode'], model_3.tvalues['allocation_niicode'], model_5.tvalues['allocation_niicode']]
    })
    return results



In [ ]:
# Create a list of the 5 dataframes with labels
df_list = [('df_all', df_all), ('df_se', df_se), ('df_hs', df_hs), ('df_bio', df_bio), ('df_int', df_int)]

# Create an empty list to store the results
results_list = []

# Iterate over the dataframes and apply the function to each of them
for label, df in df_list:
    results = fit_linear_regression(df, label)
    results_list.append(results)

# Concatenate the results into a single dataframe
final_results = pd.concat(results_list, ignore_index=True)

# Print the final results
display(final_results)

### Data Viz

In [ ]:
df_targeting = df_all
title_string = 'All Academic Fields'

# Create separate dataframes for treatment and control groups
treatment = df_targeting[df_targeting['allocation_niicode'] == 1]
control = df_targeting[df_targeting['allocation_niicode'] == 0]

print("-----------ave_year--------------")
# Fit linear regression for treatment group
re_treat = smf.ols(formula="ave_year ~ startfiscalyear", data=treatment).fit()
re_cont = smf.ols(formula="ave_year ~ startfiscalyear", data=control).fit()
# Plot RDD
sns.set(style="whitegrid")
sns.lmplot(x="startfiscalyear", y="ave_year", data=df_targeting, hue="allocation_niicode", ci=None, height=6, aspect=1.5, truncate=True)
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("ave 5", fontsize = 14)
plt.show()
# Add regression line for treatment group
x_treat = treatment['startfiscalyear']
y_treat = re_treat.predict(treatment)
plt.plot(x_treat, y_treat, color="red")
# Add regression line for control group
x_cont = control['startfiscalyear']
y_cont = re_cont.predict(control)
plt.plot(x_cont, y_cont, color="blue")
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize = 14)
plt.show()
print("-----------ave_3--------------")
# Fit linear regression for treatment group
re_treat = smf.ols(formula="ave_3 ~ startfiscalyear", data=treatment).fit()
re_cont = smf.ols(formula="ave_3 ~ startfiscalyear", data=control).fit()
# Plot RDD
sns.set(style="whitegrid")
sns.lmplot(x="startfiscalyear", y="ave_3", data=df_targeting, hue="allocation_niicode", ci=None, height=6, aspect=1.5, truncate=True)
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("ave 5", fontsize = 14)
plt.show()
# Add regression line for treatment group
x_treat = treatment['startfiscalyear']
y_treat = re_treat.predict(treatment)
plt.plot(x_treat, y_treat, color="red")
# Add regression line for control group
x_cont = control['startfiscalyear']
y_cont = re_cont.predict(control)
plt.plot(x_cont, y_cont, color="blue")
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize = 14)
plt.show()

print("-----------ave_5--------------")

# Fit linear regression for treatment group
re_treat = smf.ols(formula="ave_5 ~ startfiscalyear", data=treatment).fit()
re_cont = smf.ols(formula="ave_5 ~ startfiscalyear", data=control).fit()
# Plot RDD
sns.set(style="whitegrid")
sns.lmplot(x="startfiscalyear", y="ave_5", data=df_targeting, hue="allocation_niicode", ci=None, height=6, aspect=1.5, truncate=True)
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("ave 5", fontsize = 14)
plt.show()
# Add regression line for treatment group
x_treat = treatment['startfiscalyear']
y_treat = re_treat.predict(treatment)
plt.plot(x_treat, y_treat, color="red")
# Add regression line for control group
x_cont = control['startfiscalyear']
y_cont = re_cont.predict(control)
plt.plot(x_cont, y_cont, color="blue")
# Set plot title and axis labels
plt.title("Regression Discontinuity: Before and After the Cutoff: \n" + title_string,  fontsize = 14)
plt.xlabel("Start Fiscal Year of the Project",fontsize = 14)
plt.ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize = 14)
plt.show()
